In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import pickle
from torch.nn.functional import pad, one_hot

from torch.utils.data import DataLoader
from matplotlib import pyplot as plt

import Transformer
from Transformer import TransformerClassifier, MyDataset, create_padding_mask

Combine smaller files

In [2]:
# ins = []
# for i in range (1,16):
#     name = "Train_Data_50_" +str(i)
#     with open(name,"rb") as file:
#         ins.append(pickle.load(file))

# out = pd.concat([x for x in ins])

# with open("Train_Data_m","wb") as file:
#         pickle.dump(out,file)

In [3]:
# name = "Train_Data_m"
# with open(name,"rb") as file:
#     raw_data = pickle.load(file)

# raw_data.shape

Create DataLoader

In [4]:
# Import Data
name = "Train_Data_m"
with open(name,"rb") as file:
    raw_data = pickle.load(file)


# Pad Data
data = raw_data["System_EWS"]
seq = [torch.from_numpy(run).float() for run in data]
#seq_trun = [torch.from_numpy(run[:,:6]).float() for run in data]

seq_padded = []
max_length = 650
for run in seq:
    pad_amount = max_length - run.shape[0]
    run_padded = pad(run, (0, 0, pad_amount, 0))
    seq_padded.append(run_padded)
input_data = torch.stack(seq_padded)

# Create one-hot labels
labels = [torch.tensor(label) for label in raw_data["null"]]
labels_t = torch.stack(labels)
labels_oh = one_hot(labels_t).float()


dataset = MyDataset(input_data, labels_oh)
batch_size = 50
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [2]:
input_data = torch.load("Test_Data")
labels_oh = torch.load("Test_Labels")
classes = torch.argmax(labels_oh,dim=1)

In [3]:
input_data.shape, labels_oh.shape

(torch.Size([62511, 600, 12]), torch.Size([62511, 2]))

Import Test Data

In [4]:
dataset = MyDataset(input_data, classes)
batch_size = 125
dataloader = DataLoader(dataset, batch_size, shuffle=True)

Initialize Model and Select Parameters

In [5]:
dim_feedforward = 128
input_dim = 12  # Number of features
d_model = 128  # Transformer model dimension
num_heads = 8   # Number of attention heads
num_layers = 6  # Number of transformer layers
#num_classes = 2  # Number of classes for classification
dropout = 0.1   # Dropout rate

model = TransformerClassifier(input_dim, d_model, num_heads, num_layers, dim_feedforward, dropout)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()

In [6]:
print(model)

TransformerClassifier(
  (batch_norm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (projection): Linear(in_features=12, out_features=64, bias=True)
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.15, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-5): 6 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=64, bias=True)
        (dropout): Dropout(p=0.15, inplace=False)
        (linear2): Linear(in_features=64, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.15, inplace=False)
        (dropout2): Dropout(p=0.15, inplace=False)
      )
    )
  )
  (classificati

Train Model

In [7]:
epochs = 10 # number of epochs

losses = []

for epoch in range(1,epochs+1):
    for batch_data, batch_labels in dataloader:
        # Forward pass with masking
        output = model(batch_data)
        # print(f"output shape: {output.shape},output: {output}")  # Output will have shape: [batch_size, num_classes]
        # print(f"labels: {batch_labels}")
        # Example of using CrossEntropyLoss for training
        loss = criterion(output, batch_labels)
        print(f"Loss: {loss.item()}")
        losses.append(loss.item())
        optimizer.zero_grad()
        
        loss.backward()
        optimizer.step()

Loss: 0.7675912976264954
Loss: 0.9038965106010437
Loss: 0.5319786667823792
Loss: 0.666491687297821
Loss: 0.5262066125869751
Loss: 0.513166606426239
Loss: 0.47399261593818665
Loss: 0.5627514123916626
Loss: 0.5417015552520752
Loss: 0.34526893496513367
Loss: 0.49222660064697266
Loss: 0.4690268337726593
Loss: 0.4345008134841919
Loss: 0.4548071324825287
Loss: 0.4531703591346741
Loss: 0.36333122849464417
Loss: 0.3870103359222412
Loss: 0.4679955244064331
Loss: 0.22294801473617554
Loss: 0.5368009209632874
Loss: 0.3519337475299835
Loss: 0.34666764736175537
Loss: 0.455465704202652
Loss: 0.2589505612850189
Loss: 0.30897969007492065
Loss: 0.35583531856536865
Loss: 0.2720887064933777
Loss: 0.41682547330856323
Loss: 0.38769474625587463
Loss: 0.2817652225494385
Loss: 0.332571804523468
Loss: 0.3067631423473358
Loss: 0.2773068845272064
Loss: 0.27955546975135803
Loss: 0.2416948676109314
Loss: 0.3283304274082184
Loss: 0.2059617042541504
Loss: 0.2941317558288574
Loss: 0.28804218769073486
Loss: 0.478790700

torch.Size([62511])

In [1]:
# test_batch = input_data[:5000,:,:]
# test_labels = classes[:5000]
# raw_preds = model(batch_data)

NameError: name 'input_data' is not defined